In [66]:
import asyncio
import load_env
import src.graphql.models.recipe_models_autogen as rma
from sqlalchemy import select
from src.graphql.db.session import get_session
from src.graphql.scalars.recipe_scalar import NEI_Item, NEI_Fluid, NEI_Recipe_Dimensions, NEI_GT_Recipe
from src.graphql.resolvers.recipe_resolver import _getNEIRecipe, _getNEIGTRecipe

In [69]:
# Test joins
recipe_ids = ['r~--eEYPHsN5GqWygrEHan4A==']

# stmt = select(rma.Recipe.id, rma.RecipeType.category) \
#         .join(rma.Recipe, rma.Recipe.recipe_type_id == rma.RecipeType.id) \
#         .filter(rma.Recipe.id.in_(recipe_ids)) \
#         .where(rma.RecipeType.category == 'gregtech')
stmt = select(rma.RecipeItemGroup.id) \
        .join(rma.RecipeItemGroup, rma.RecipeItemGroup.item_inputs_id == rma.ItemGroupItemStacks.item_group_id) \

print(stmt)

SELECT recipe.id, recipe_type.category 
FROM recipe_type JOIN recipe ON recipe.recipe_type_id = recipe_type.id 
WHERE recipe.id IN (__[POSTCOMPILE_id_1]) AND recipe_type.category = :category_1


In [70]:
def _prepORMDict(item, rename={}, exclude=[], include={}):
    d = dict(item.__dict__)
    d.pop('_sa_instance_state')

    for k, v in rename.items():
        d[v] = d.pop(k)
    for k in exclude:
        d.pop(k)
    d.update(include)
    
    return d


async with get_session() as session:
    rows = await session.execute(stmt)
    awaitables = [_getNEIGTRecipe(session, recipe_id) for recipe_id, _ in rows]
    gt_recipes = await asyncio.gather(*awaitables)

print(gt_recipes)

    # gt_recipes = []
    # for r in rows:
    #     gtRecipeORM, recipeTypeORM = r
    #     findict = _prepORMDict(
    #         recipeTypeORM,
    #         rename={
    #             'type': 'localized_machine_name',
    #         },
    #         exclude = [
    #             'category', # They're all "gregtech"
    #             'id'
    #         ],
    #     )
    #     findict.update(_prepORMDict(
    #         gtRecipeORM,
    #         rename = {
    #             'duration': 'duration_ticks',
    #         },
    #         exclude = ['id'],
    #     ))
    #     for single_type in ['item', 'fluid']:
    #         for direction in ['input', 'output']:
    #             findict[f'{single_type}_{direction}_dims'] = NEI_Recipe_Dimensions(
    #                 height = findict.pop(f'{single_type}_{direction}_dimension_height'),
    #                 width = findict.pop(f'{single_type}_{direction}_dimension_width'),
    #             )
    #     findict['base_recipe'] = await _getNEIRecipe(session, recipe_id)

    #     gt_recipes.append(
    #         NEI_GT_Recipe(**findict)
    #     )

    # print(result)
    # for r in result:
    #     print(r.__dict__)

    # print(gt_recipes)
    # print(len(gt_recipes))

[NEI_GT_Recipe(recipe_id='r~--eEYPHsN5GqWygrEHan4A==', base_recipe=NEI_Base_Recipe(recipe_id='r~--eEYPHsN5GqWygrEHan4A==', input_items=[NEI_Item(id='i~gregtech~gt.metaitem.01~23019', item_id=7197, position=1, stack_size=1, image_file_path='item/gregtech/gt.metaitem.01~23019.png', internal_name='gt.metaitem.01', item_damage=23019, localized_name='Aluminium Rod', max_damage=0, max_stack_size=64, mod_id='gregtech', nbt='', tooltip='Aluminium Rod (#7197/23019)\\u000agregtech:gt.metaitem.01\\u000aAl', unlocalized_name='gt.metaitem.01.23019', input=True, output_probability=-1.0), NEI_Item(id='i~gregtech~gt.metaitem.01~23355', item_id=7197, position=0, stack_size=1, image_file_path='item/gregtech/gt.metaitem.01~23355.png', internal_name='gt.metaitem.01', item_damage=23355, localized_name='Magnetic Steel Rod', max_damage=0, max_stack_size=64, mod_id='gregtech', nbt='', tooltip='Magnetic Steel Rod (#7197/23355)\\u000agregtech:gt.metaitem.01\\u000aFe\\u2085\\u2080C', unlocalized_name='gt.metaite